In [68]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
import argparse
import igraph
from igraph import Graph, EdgeSeq
np.random.seed(0)

# System Dynamics

In [69]:
# System Dynamics
A = np.array([
    [1, 1],
    [0, 1]
    ])
B = np.array([
    [1/2],
    [1]
    ])
C = np.eye(2)
D = np.zeros((2,1))

def step(s, u):
    return np.dot(A,s) + np.dot(B,u)


# States
x = np.zeros(2)
x_min = -5
x_max = -x_min
v_min = -1
v_max = -v_min
s_f = np.array([[0],[0]])
def check_bounds(s):
    return np.abs(s[0,0]) < x_max and np.abs(s[1,0]) < v_max
def goal_reached(s, s_f, dx=0.01, dv=0.01):
    return (np.abs(s[0,0] - s_f[0,0]) < dx) and  (np.abs(s[1,0] - s_f[1,0]) < dv)

# Randomized Initial State

In [70]:
# Randomized Initial State
s_i = np.zeros((2,1)) 
s_i[0,0] = np.random.uniform(x_min, x_max)
s_i[1,0] = np.random.uniform(v_min, v_max)
print("s_i:\n", s_i)

s_i:
 [[0.48813504]
 [0.43037873]]


# Cost Function

In [71]:
Q = np.eye(2)
R = 100

# Dynamic Programming Solution

In [72]:
P_now = Q
K_now = 0
P_prev = np.zeros((2,2))
K_prev = 0

while not np.allclose(P_now,P_prev):
    P_now = np.copy(P_prev)
    K_now = np.copy(K_prev)
    P_prev = Q + np.dot(np.transpose(K_now),np.dot(R,K_now)) + np.transpose(A - np.dot(B, K_now))*P_now*(A - np.dot(B, K_now))
    K_prev = np.dot(np.linalg.inv(R + np.dot(np.transpose(B), np.dot(P_now, B))), np.dot(np.transpose(B), np.dot(P_now, A)))


In [73]:
print("P:\n", P_prev)
print("K:\n", K_prev)

P:
 [[20.5203192   2.2317375 ]
 [ 2.2317375  16.00610056]]
K:
 [[0.10125687 0.24004417]]


In [74]:
def demo(K):
    S = [s_i]
    E = []
    while True:
        s_new = step(S[-1],np.dot(-K,S[-1]))
        S.append(s_new)
        E.append((S[-2],S[-1]))
        if goal_reached(s_new, s_f):
            return S, E
S, E = demo(K_prev)
    

# Plotting

In [75]:
Xe = []
Ye = []
for edge in E:
    Xe+=[edge[0][0,0],edge[1][0,0], None]
    Ye+=[edge[0][1,0],edge[1][1,0], None]

Xn = [x[0,0] for x in S]
Yn = [x[1,0] for x in S]
Xi = [s_i[0,0]]
Yi = [s_i[1,0]]
Xf = [s_f[0,0]]
Yf = [s_f[1,0]]

fig = go.Figure()
fig.add_trace(go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                         name='Edges',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   ))
fig.add_trace(go.Scatter(x=Xn,
                  y=Yn,
                  mode='markers',
                  name='Vertices',
                  marker=dict(symbol='circle-dot',
                                size=2,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  hoverinfo='text',
                  opacity=0.8
                  ))
fig.add_trace(go.Scatter(x=Xi,
                  y=Yi,
                  mode='markers',
                  name='Start',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#f67c07',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  hoverinfo='text',
                  opacity=0.8
                  ))

fig.add_trace(go.Scatter(x=Xf,
                  y=Yf,
                  mode='markers',
                  name='Goal',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#54ca41',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  hoverinfo='text',
                  opacity=0.5
                  ))
fig.update_layout(title={'text':"LQR Without Noise",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title="Position", yaxis_title="Velocity")
fig.show()